In [79]:
import shapefile as sf
import numpy as np

myfile = sf.Reader("/g/data1a/ge3/AEM_Model/3D_AEM_model_V3.shp")

myshaperecord = myfile.shapeRecords()

myrecord=(x.record for x in myshaperecord)

from itertools import cycle, compress



with open('/g/data1a/ge3/yuhang/AEM_predict.txt','w') as fw:
    for r in myrecord:
        if r[1]!='Surface':         
            r[1]=int(r[1])         
            np.savetxt(fw, np.array(r).reshape(1,-1), fmt='%s')

In [2]:
import numpy as np
import scipy as sp
import scipy.spatial
import itertools as it


fr = open('AEM.txt')

lines = np.array([map(float, line.split()) for line in fr])


#extract label, drop label and unique-id
label = lines[:,-2]
lines = np.delete(lines,0,1)
lines = np.delete(lines,-2,1)

#whitening transform
lines_whitened = (lines-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))

#build index
data_index=scipy.spatial.KDTree(lines_whitened)

In [3]:
fr_test = open('/g/data1a/ge3/yuhang/AEM_predict.txt')

lines_test = np.array([map(float, line.split()) for line in fr_test])

#extract xyz, drop xyz and unique-id
xyz_test = lines_test[:,10:13]


lines_test = np.delete(lines_test,[0,1,10,11,13],1)

#whitening transform
lines_whitened = (lines_test-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))




from multiprocessing import Pool



def minibatch(bid,bsize=1000):
    return data_index.query(lines_whitened[bid*bsize:(bid+1)*bsize],1)

p=Pool(10)
prediction = p.map(minibatch,range(1268))

label_predicted = label[map(int,np.hstack(prediction)[1,])]


label_predicted_taged=zip(xyz_test,label_predicted)

with open('/g/data1a/ge3/yuhang/AEM_predict_result.txt','w') as fw:
    for r in label_predicted_taged:    
            np.savetxt(fw, np.array(r).reshape(1,-1), fmt='%s')

ValueError: operands could not be broadcast together with shapes (1267680,12) (10,) 

In [1]:

#whitening transform
lines_whitened = (lines_test-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))




from multiprocessing import Pool



def minibatch(bid,bsize=1000):
    return data_index.query(lines_whitened[bid*bsize:(bid+1)*bsize],1)

p=Pool(10)
prediction = p.map(minibatch,range(1268))

label_predicted = label[map(int,np.hstack(prediction)[1,])]

label_predicted_taged=zip(xyz_test,label_predicted)


with open('/g/data1a/ge3/yuhang/AEM_predict_result.txt','w') as fw:
    for r in label_predicted_taged:    
            np.savetxt(fw, np.array(r).reshape(1,-1), fmt='%s')

NameError: name 'lines_test' is not defined

In [31]:
import sklearn.decomposition as sd
import numpy as np
import re

fr_result = open('/g/data1a/ge3/yuhang/AEM_predict_result.txt')

lines_result = [map(float,re.findall(r"[^\[\], ]+",line)) for line in fr_result]




xy = [r[0:2] for r in lines_result]

pc = sd.PCA(n_components=1).fit(xy)

xy=pc.transform(xy)

xy=xy-xy.min(axis=0)

#xy_label = zip(xy.astype(int),[r[3] for r in lines_result])

In [374]:
#################latest

import sklearn.decomposition as sd
import numpy as np
import re

fr_result = open('/g/data1a/ge3/yuhang/AEM_predict_result.txt')

lines_result = np.array([map(float,re.findall(r"[^\[\], ]+",line)) for line in fr_result])

#lines_result=lines_result[:30000]

lines_result[:,2]=np.round(lines_result[:,2]/1)

height = len(np.unique(lines_result[:,0:2],axis=0))
width = len(np.unique(lines_result[:,2],axis=0))

shrinkage = 1

output = np.zeros([width/shrinkage+1,height/shrinkage+1],dtype=np.uint8)


pca=sd.PCA(n_components=1)
pca.fit(lines_result[:,0:2])
newx=pca.transform(lines_result[:,0:2])
x_dic=set([x[0] for x in newx])
x_dic=dict(zip(np.sort(list(x_dic)),np.array(range(len(x_dic)))))

y_dic=set(lines_result[:,2])
y_dic=dict(zip(np.sort(list(y_dic)),np.array(range(len(y_dic)))))

for r in lines_result:
    x=x_dic[pca.transform(r[0:2].reshape(1,2))[0,0]]/shrinkage
    y=width-y_dic[r[2]]/shrinkage      
    output[y,x] = r[3]*28
    #print(x,' ',y)
   

for i in range(height):
    current=output[0,height]
    for j in range(width,0):
        if current==0 and output[j,i]==0:
            output[j,i]=0
        elif output[j,i]>0:
            current=1
        else:
            continue
            
            
            
    
for i in range(height):
    current=0
    for j in range(width):
        if output[j,i]==0:
            output[j,i]=current
        else:
            current=output[j,i]
            
    

            
#import scipy.misc as smp
from PIL import Image
import scipy.ndimage as ndimage

#img = smp.toimage(output)

#img = ndimage.maximum_filter(output, size=8)

img = Image.fromarray(output)

img.save("/g/data1a/ge3/yuhang/prediction.png","PNG")

In [388]:

for r in lines_result:
    x=x_dic[pca.transform(r[0:2].reshape(1,2))[0,0]]/shrinkage
    y=width-y_dic[r[2]]/shrinkage      
    output[y,x] = r[3]*28
    #print(x,' ',y)
   

for i in range(height):
    current=output[0,height]
    for j in range(width,0):
        if current==0 and output[j,i]==0:
            output[j,i]=0
        elif output[j,i]>0:
            current=1
        else:
            output[j,i]=1
            
            
            
    
for i in range(height):
    current=0
    for j in range(width):
        if output[j,i]==1:
            output[j,i]=current
        else:
            current=output[j,i]
            
    

            
#import scipy.misc as smp
from PIL import Image
import scipy.ndimage as ndimage

#img = smp.toimage(output)

#img = ndimage.maximum_filter(output, size=8)

img = Image.fromarray(output)

img.save("/g/data1a/ge3/yuhang/prediction.png","PNG")

In [589]:
o2=np.copy(output)

for i in range(0,200):
    for j in range(30000,32600):
        o2[i,j]=255
        

        
img = Image.fromarray(o2)

img.save("/g/data1a/ge3/yuhang/test.png","PNG")        

In [578]:
width

1293

In [378]:

    
for i in height:
    current=0
    for j in width:
        if output[j,i]=0:
            output[j,i]=current
        else:
            current=output[j,i]
    
#import scipy.misc as smp
from PIL import Image
import scipy.ndimage as ndimage

#img = smp.toimage(output)

#img = ndimage.maximum_filter(output, size=8)

img = Image.fromarray(img)

img.save("/g/data1a/ge3/yuhang/prediction.png","PNG")

SyntaxError: invalid syntax (<ipython-input-378-e16773a76b00>, line 6)

In [456]:
output[800,32516:33000]

array([252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
       252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
       252, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 22

In [486]:
output[556,29126:29516]

array([196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

In [525]:
output[600,33663:33800]

array([  0, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196, 196,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0], dtype=uint8)

In [518]:
output[599,33600:33800]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=uint8)

In [601]:
lines_result[0:30]

array([[ 3.4793180e+05,  7.3396325e+06,  5.8200000e+02,  4.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.7700000e+02,  4.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.7300000e+02,  4.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.6800000e+02,  4.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.6200000e+02,  4.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.5600000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.4900000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.4200000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.3400000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.2500000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.1500000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  5.0400000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  4.9200000e+02,  8.0000000e+00],
       [ 3.4793180e+05,  7.3396325e+06,  4.7900000e

In [600]:
lines_result[10030:10060]

array([[3.5034330e+05, 7.3426745e+06, 5.8800000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.7700000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.6600000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.5200000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.3800000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.2200000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 5.0400000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 4.8500000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 4.6400000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 4.4000000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 4.1500000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 3.8700000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 3.5500000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+06, 3.2100000e+02, 8.0000000e+00],
       [3.5034330e+05, 7.3426745e+

In [594]:
y_dic[lines_result[29]]

KeyError: -39.76

In [603]:
for key, value in x_dic.iteritems:
    if value==2517:
        print key

TypeError: 'builtin_function_or_method' object is not iterable

In [591]:
for i in range(len(newx)):
    if newx[i]==-132993.9898328744:
        print i


300000
300001
300002
300003
300004
300005
300006
300007
300008
300009
300010
300011
300012
300013
300014
300015
300016
300017
300018
300019
300020
300021
300022
300023
300024
300025
300026
300027
300028
300029


In [605]:
for key, value in x_dic.iteritems():
    if value==2517:
        print key

-223517.915735603


In [606]:
for i in range(len(newx)):
    if newx[i]==-223517.915735603:
        print i

75510
75511
75512
75513
75514
75515
75516
75517
75518
75519
75520
75521
75522
75523
75524
75525
75526
75527
75528
75529
75530
75531
75532
75533
75534
75535
75536
75537
75538
75539


In [626]:
for key, value in x_dic.iteritems():
    if value==72:
        print key

-248879.18133036926


In [627]:
for i in range(len(newx)):
    if newx[i]==-248879.18133036926:
        print i

2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189


In [565]:
output[500:700,40000]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 196, 196, 196, 196, 196, 196, 196, 196,
       196, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 22

In [610]:
lines_test[75540:75570]

array([[0.0000000e+00, 1.6280500e+05, 4.8800000e-03, 5.1900000e-03,
        4.9400000e-03, 6.5900000e-03, 4.8800000e-03, 4.0000000e+00,
        1.5881000e+00, 0.0000000e+00, 3.5287450e+05, 7.3657266e+06,
        6.9524000e+02, 0.0000000e+00, 2.0000000e+00],
       [0.0000000e+00, 1.6280500e+05, 6.9600000e-03, 6.5900000e-03,
        7.0100000e-03, 1.1200000e-02, 6.9600000e-03, 4.4000000e+00,
        1.5722200e+00, 0.0000000e+00, 3.5287450e+05, 7.3657266e+06,
        6.9104000e+02, 0.0000000e+00, 2.0000000e+00],
       [0.0000000e+00, 1.6280500e+05, 8.2100000e-03, 7.9200000e-03,
        8.2000000e-03, 1.7600000e-02, 8.2100000e-03, 4.8400000e+00,
        1.5565000e+00, 0.0000000e+00, 3.5287450e+05, 7.3657266e+06,
        6.8642000e+02, 0.0000000e+00, 2.0000000e+00],
       [0.0000000e+00, 1.6280500e+05, 8.3500000e-03, 9.1400000e-03,
        8.3300000e-03, 2.3300000e-02, 8.3500000e-03, 5.3200000e+00,
        1.5409300e+00, 0.0000000e+00, 3.5287450e+05, 7.3657266e+06,
        6.8134000e+02,

In [613]:
lines_test[0:30]

array([[ 0.0000000e+00,  1.6028700e+05,  8.6200000e-04,  1.9000000e-03,
         1.1400000e-03,  1.7600000e-03,  8.6200000e-04,  4.0000000e+00,
         3.3773100e+00,  5.6622500e-03,  3.4793180e+05,  7.3396325e+06,
         5.8158000e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6028700e+05,  9.9600000e-04,  2.8800000e-03,
         1.3200000e-03,  3.7600000e-03,  9.9600000e-04,  4.4000000e+00,
         3.3435300e+00,  5.6622500e-03,  3.4793180e+05,  7.3396325e+06,
         5.7738000e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6028700e+05,  1.3900000e-03,  4.3200000e-03,
         1.7900000e-03,  9.7700000e-03,  1.3900000e-03,  4.8400000e+00,
         3.3101000e+00,  5.6622500e-03,  3.4793180e+05,  7.3396325e+06,
         5.7276000e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6028700e+05,  2.5100000e-03,  6.3700000e-03,
         3.0100000e-03,  2.3900000e-02,  2.5100000e-03,  5.3200000e+00,
         3.2770000e+00,  5.6622500e-0

In [631]:
lines_test[2100]

array([0.0000000e+00, 1.6035700e+05, 5.7800000e-02, 4.1100000e-02,
       5.6300000e-02, 6.0800000e-02, 5.7800000e-02, 4.0000000e+00,
       2.5137800e+00, 1.4959400e-04, 3.4857410e+05, 7.3402499e+06,
       5.9558000e+02, 0.0000000e+00, 1.0000000e+00])

In [630]:
lines_test[2130]

array([0.0000000e+00, 1.6035800e+05, 5.8400000e-02, 4.2400000e-02,
       5.6900000e-02, 6.1300000e-02, 5.8400000e-02, 4.0000000e+00,
       2.4898200e+00, 1.5874800e-04, 3.4858150e+05, 7.3402591e+06,
       5.9580000e+02, 0.0000000e+00, 1.0000000e+00])

In [629]:
lines_test[2160]

array([0.0000000e+00, 1.6035900e+05, 5.8000000e-02, 4.3100000e-02,
       5.6500000e-02, 6.1300000e-02, 5.8000000e-02, 4.0000000e+00,
       2.3883700e+00, 1.1857600e-04, 3.4858900e+05, 7.3402684e+06,
       5.9626000e+02, 0.0000000e+00, 1.0000000e+00])

In [632]:
lines_test[2190]

array([0.0000000e+00, 1.6036000e+05, 5.7700000e-02, 4.4000000e-02,
       5.6300000e-02, 6.1400000e-02, 5.7700000e-02, 4.0000000e+00,
       2.2446000e+00, 1.8723600e-04, 3.4859650e+05, 7.3402777e+06,
       5.9673000e+02, 0.0000000e+00, 1.0000000e+00])

In [625]:
output[800,71:200]

array([252, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 252, 252,
       252, 252, 252, 252, 252, 252, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224, 224,
       224, 224, 224, 224, 224, 224, 224, 224, 252, 252, 252, 252, 252,
       252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
       252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
       252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252,
       252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252, 252],
      dtype=uint8)

In [633]:
prediction

NameError: name 'prediction' is not defined

In [635]:
import numpy as np
import scipy as sp
import scipy.spatial
import itertools as it


fr = open('AEM.txt')

lines = np.array([map(float, line.split()) for line in fr])


#extract label, drop label and unique-id
label = lines[:,-2]
lines = np.delete(lines,0,1)
lines = np.delete(lines,-2,1)

#whitening transform
lines_whitened = (lines-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))

#build index
data_index=scipy.spatial.KDTree(lines_whitened)

fr_test = open('/g/data1a/ge3/yuhang/AEM_predict.txt')

lines_test = np.array([map(float, line.split()) for line in fr_test])

#extract xyz, drop xyz and unique-id
xyz_test = lines_test[:,10:13]


lines_test = np.delete(lines_test,[0,1,10,11,13],1)

#whitening transform
lines_whitened = (lines_test-lines.min(axis=0))/(lines.max(axis=0)-lines.min(axis=0))




from multiprocessing import Pool



def minibatch(bid,bsize=1000):
    return data_index.query(lines_whitened[bid*bsize:(bid+1)*bsize],1)

p=Pool(10)
prediction = p.map(minibatch,range(1268))

#label_predicted = label[map(int,np.hstack(prediction)[1,])]


In [636]:
prediciton=map(int,np.hstack(prediction)[1,])

In [691]:
prediciton[2157:2160]

[147, 119, 149]

In [692]:
label[[147, 119, 149]]

array([9., 8., 9.])

In [690]:
fr_test = open('/g/data1a/ge3/yuhang/AEM_predict.txt')

lines_test = np.array([map(float, line.split()) for line in fr_test])
lines_test[2157:2160]

array([[ 0.0000000e+00,  1.6035800e+05,  3.2500000e-03,  3.8400000e-03,
         3.7800000e-03,  3.4500000e-02,  3.2500000e-03,  5.2440000e+01,
         1.8981000e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         8.7200000e+01,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  3.7900000e-03,  3.5700000e-03,
         3.9900000e-03,  4.2400000e-02,  3.7900000e-03,  5.7680000e+01,
         1.8791200e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         3.2140000e+01,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  4.9200000e-03,  3.3800000e-03,
         4.6200000e-03,  5.3200000e-02,  4.9200000e-03,  5.7680000e+01,
         1.8603200e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
        -2.5540000e+01,  0.0000000e+00,  1.0000000e+00]])

In [649]:
lines_test[2151]

array([0.0000000e+00, 1.6035800e+05, 9.4000000e-03, 1.0400000e-02,
       9.0900000e-03, 6.1700000e-02, 9.4000000e-03, 2.9600000e+01,
       2.0160800e+00, 1.5874800e-04, 3.4858150e+05, 7.3402591e+06,
       3.2701000e+02, 0.0000000e+00, 1.0000000e+00])

In [650]:
lines_test[2152]

array([0.0000000e+00, 1.6035800e+05, 4.8500000e-03, 8.9200000e-03,
       5.0600000e-03, 6.3400000e-02, 4.8500000e-03, 3.2560000e+01,
       1.9959200e+00, 1.5874800e-04, 3.4858150e+05, 7.3402591e+06,
       2.9593000e+02, 0.0000000e+00, 1.0000000e+00])

In [651]:
lines_test[2181]

array([0.0000000e+00, 1.6035900e+05, 9.3000000e-03, 1.0400000e-02,
       9.0100000e-03, 6.0900000e-02, 9.3000000e-03, 2.9600000e+01,
       1.9339300e+00, 1.1857600e-04, 3.4858900e+05, 7.3402684e+06,
       3.2747000e+02, 0.0000000e+00, 1.0000000e+00])

In [657]:
lines_test[2331]

array([0.0000000e+00, 1.6036400e+05, 8.6300000e-03, 1.0100000e-02,
       8.4300000e-03, 5.7900000e-02, 8.6300000e-03, 2.9600000e+01,
       1.4890100e+00, 2.6539700e-04, 3.4862580e+05, 7.3403148e+06,
       3.2969000e+02, 0.0000000e+00, 1.0000000e+00])

In [658]:
lines_test[2361]

array([0.0000000e+00, 1.6036500e+05, 8.5000000e-03, 1.0000000e-02,
       8.3200000e-03, 5.7900000e-02, 8.5000000e-03, 2.9600000e+01,
       1.3864300e+00, 2.4884600e-04, 3.4863300e+05, 7.3403241e+06,
       3.3006000e+02, 0.0000000e+00, 1.0000000e+00])

In [659]:
lines_test[2391]

array([0.0000000e+00, 1.6036600e+05, 8.3400000e-03, 9.9700000e-03,
       8.1800000e-03, 5.7600000e-02, 8.3400000e-03, 2.9600000e+01,
       1.2736200e+00, 1.8158100e-04, 3.4864010e+05, 7.3403335e+06,
       3.3027000e+02, 0.0000000e+00, 1.0000000e+00])

In [667]:
lines_test[2189]

array([ 0.0000000e+00,  1.6035900e+05,  4.7600000e-03,  3.2700000e-03,
        4.4700000e-03,  5.2400000e-02,  4.7600000e-03,  5.7680000e+01,
        1.7845200e+00,  1.1857600e-04,  3.4858900e+05,  7.3402684e+06,
       -2.5080000e+01,  0.0000000e+00,  1.0000000e+00])

In [668]:
lines_result[2189]

array([ 3.4858900e+05,  7.3402684e+06, -2.5000000e+01,  8.0000000e+00])

In [685]:
lines_test[2155:2160]

array([[ 0.0000000e+00,  1.6035800e+05,  2.5800000e-03,  4.9100000e-03,
         3.2300000e-03,  3.3700000e-02,  2.5800000e-03,  4.3340000e+01,
         1.9366400e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         1.8276000e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  2.9000000e-03,  4.2500000e-03,
         3.5700000e-03,  3.1500000e-02,  2.9000000e-03,  4.7670000e+01,
         1.9172700e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         1.3725500e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  3.2500000e-03,  3.8400000e-03,
         3.7800000e-03,  3.4500000e-02,  3.2500000e-03,  5.2440000e+01,
         1.8981000e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         8.7200000e+01,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  3.7900000e-03,  3.5700000e-03,
         3.9900000e-03,  4.2400000e-02,  3.7900000e-03,  5.7680000e+01,
         1.8791200e+00,  1.5874800e-0

In [689]:
x=2130
y=0

lines_result[x+y:x+y+30]

array([[ 3.4858150e+05,  7.3402591e+06,  5.9600000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.9200000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.8700000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.8200000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.7600000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.7000000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.6300000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.5600000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.4800000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.3900000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.2900000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.1800000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  5.0600000e+02,  8.0000000e+00],
       [ 3.4858150e+05,  7.3402591e+06,  4.9300000e

In [688]:
lines_test[2155:2160]

array([[ 0.0000000e+00,  1.6035800e+05,  2.5800000e-03,  4.9100000e-03,
         3.2300000e-03,  3.3700000e-02,  2.5800000e-03,  4.3340000e+01,
         1.9366400e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         1.8276000e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  2.9000000e-03,  4.2500000e-03,
         3.5700000e-03,  3.1500000e-02,  2.9000000e-03,  4.7670000e+01,
         1.9172700e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         1.3725500e+02,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  3.2500000e-03,  3.8400000e-03,
         3.7800000e-03,  3.4500000e-02,  3.2500000e-03,  5.2440000e+01,
         1.8981000e+00,  1.5874800e-04,  3.4858150e+05,  7.3402591e+06,
         8.7200000e+01,  0.0000000e+00,  1.0000000e+00],
       [ 0.0000000e+00,  1.6035800e+05,  3.7900000e-03,  3.5700000e-03,
         3.9900000e-03,  4.2400000e-02,  3.7900000e-03,  5.7680000e+01,
         1.8791200e+00,  1.5874800e-0

In [693]:
label

array([8., 8., 8., ..., 8., 8., 8.])

In [694]:
label.max()

9.0